In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [2]:
import json


root_path = 'gdrive/MyDrive/Colab Notebooks/lab9-nlp/'
with open(root_path+'intents.json') as json_data:
    intents = json.load(json_data)

In [3]:
print(intents)

{'intents': [{'tag': 'greeting', 'patterns': ['Hi', 'How are you', "What's up?", 'Hello', 'Good day'], 'responses': ['Hello, how are you today?', 'Good to see you again. How are you?', 'Hi there, how are you?'], 'context_set': 'howareyou'}, {'tag': 'greeting_good', 'patterns': ["I'm fine", "I'm pretty good", "I'm happy today"], 'responses': ["I'm very happy to hear that!", "Yay, thats's great news!"], 'context_filter': 'howareyou'}, {'tag': 'greeting_bad', 'patterns': ['Not so good', "I'm not feeling myself today", 'I am sadder than ever.'], 'responses': ['Oh, I know jokes to make you feel better.', 'Do you want me to tell you a joke to feel better?'], 'context_filter': 'howareyou'}, {'tag': 'joke', 'patterns': ['ok, tell me a joke', 'tell me a joke', 'do you know any jokes'], 'responses': ['What did the wall tell to the other? See you at the corner!', "When Chuck Norris cuts an onion, he doesn't cry, the onion cries!"]}, {'tag': 'goodbye', 'patterns': ['Bye', 'See you later', 'Goodbye

In [4]:
! pip install tflearn

     |████████████████████████████████| 107 kB 7.5 MB/s 
  Created wheel for tflearn: filename=tflearn-0.5.0-py3-none-any.whl size=127299 sha256=a981d53437cd8290506a99a8fe15882c3579bed14c561575efb58b2d76ef1b0d
  Stored in directory: /root/.cache/pip/wheels/5f/14/2e/1d8e28cc47a5a931a2fb82438c9e37ef9246cc6a3774520271
Successfully built tflearn


In [5]:
import warnings
warnings.filterwarnings('ignore')

import nltk
nltk.download('punkt')
from nltk.stem.lancaster import LancasterStemmer
stemmer = LancasterStemmer()

import numpy as np
import tflearn
import tensorflow as tf
import random

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
Instructions for updating:
non-resource variables are not supported in the long term


In [6]:
words = []
classes = []
documents = []
ignore_words = ['?',"'m", "'re", "'s", ')', ',', '.', ':']  # eventual aici putem defini o lista de STOPWORDS

# pentru fiecare fraza din sabloanele corespunzatoare unei intentii
for intent in intents['intents']:
    for pattern in intent['patterns']:
        # tokenizare
        w = nltk.word_tokenize(pattern)
        # adaugam cuvintele la o lista globala cu toate cuvintele din texte
        words.extend(w)
        # adaugam textul tokenizat la multimea de documente
        documents.append((w, intent['tag']))
        # adaugam clasa (tipul de intentie) la lista de clase existente
        if intent['tag'] not in classes:
            classes.append(intent['tag'])

# aplicam stemming si lowercasing pentru fiecare cuvant intalnit, ignoram stopwords
words = [stemmer.stem(w.lower()) for w in words if w not in ignore_words]
words = sorted(list(set(words)))  # eliminam duplciate

classes = sorted(list(set(classes)))

print (len(documents), "documents", documents[0], documents[7])
print (len(classes), "classes", classes)
print (len(words), "unique stemmed words", words)

33 documents (['Hi'], 'greeting') (['I', "'m", 'happy', 'today'], 'greeting_good')
11 classes ['funny', 'goodbye', 'greeting', 'greeting_bad', 'greeting_good', 'joke', 'name', 'quote', 'real_therapy', 'thanks', 'therapy']
61 unique stemmed words ['a', 'am', 'any', 'ar', 'being', 'bright', 'bye', 'can', 'chatbot', 'com', 'day', 'do', 'ev', 'feel', 'fin', 'funny', 'good', 'goodby', 'hahah', 'happy', 'hello', 'help', 'hi', 'how', 'hum', 'i', 'is', 'jok', 'kind', 'know', 'lat', 'me', 'my', 'myself', 'nam', 'nee', 'not', 'of', 'ok', 'pretty', 'quot', 'real', 'sad', 'see', 'so', 'talk', 'tel', 'than', 'thank', 'that', 'therap', 'therapy', 'to', 'today', 'up', 'want', 'was', 'what', 'who', 'yo', 'you']


In [7]:
# construim datele de antrenare
training = []
output = []
# un vector de 0 de lungime egala cu numarul de clase (vom folosi reprezentarea one-hot a claselor)
output_empty = [0] * len(classes)

# bag of words pentru fiecare fraza
for doc in documents:
    # initializam bag of words
    bag = []
    # lista de tokeni pentru fraza curenta
    pattern_words = doc[0]
    # aplicam stemming
    pattern_words = [stemmer.stem(word.lower()) for word in pattern_words]
    # construim vectorul binar pentru bag of words
    for w in words:
        bag.append(1) if w in pattern_words else bag.append(0)

    # output este '1' pentru tagul corespunzator frazei si '0' pentru celelalte (one-hot)
    output_row = list(output_empty)
    output_row[classes.index(doc[1])] = 1

    training.append([bag, output_row])

print(training[1])
print(type(training))

[[0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1], [0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0]]
<class 'list'>


In [8]:
# shuffle + transformare in np.array
random.shuffle(training)
training = np.array(training)

print(training.shape)

(33, 2)


In [9]:
# feature-urile de antrenare
train_x = list(training[:,0])
# etichete/labels (ce dorim sa returneze modelul)
train_y = list(training[:,1])

In [10]:
from tensorflow.python.framework import ops
ops.reset_default_graph()  # resetam starea engine-ului TensorFlow

# definim reteaua
net = tflearn.input_data(shape=[None, len(train_x[0])])  # toti vectorii de bag-of-words au aceasta dimensiune
net = tflearn.fully_connected(net, 8) # strat feed-forward ascuns cu 8 noduri
net = tflearn.fully_connected(net, 8) # strat feed-forward ascuns cu 8 noduri
net = tflearn.fully_connected(net, len(train_y[0]), activation='softmax') # numarul de noduri output = numarul de clase
net = tflearn.regression(net)  # folosim acest strat de logistic regression pentru a extrage probabilitatile claselor

# definim modelul final
model = tflearn.DNN(net, tensorboard_dir='tflearn_logs')

# incepem antrenarea folosind coborarea pe gradient
# trecem prin model cate 8 fraze odata (batch size = 8)
model.fit(train_x, train_y, n_epoch=1000, batch_size=8, show_metric=True)
model.save('model.tflearn')

Training Step: 4999  | total loss: 0.06953 | time: 0.026s
| Adam | epoch: 1000 | loss: 0.06953 - acc: 0.9976 -- iter: 32/33
Training Step: 5000  | total loss: 0.06759 | time: 0.034s
| Adam | epoch: 1000 | loss: 0.06759 - acc: 0.9978 -- iter: 33/33
--
INFO:tensorflow:/content/model.tflearn is not in all_model_checkpoint_paths. Manually adding it.


In [11]:
import pickle
pickle.dump( {'words':words, 'classes':classes, 'train_x':train_x, 'train_y':train_y}, open( "training_data", "wb" ) )

In [12]:
data = pickle.load( open( "training_data", "rb" ) )
print(type(data))

for key in data.keys():
    print(key)

<class 'dict'>
words
classes
train_x
train_y


In [13]:
words = data['words']
classes = data['classes']
train_x = data['train_x']
train_y = data['train_y']

In [14]:
with open(root_path+'intents.json') as json_data:
    intents = json.load(json_data)

In [15]:
model.load('model.tflearn')

INFO:tensorflow:Restoring parameters from /content/model.tflearn


In [16]:
def clean_up_sentence(sentence):
    # tokenizare
    sentence_words = nltk.word_tokenize(sentence)
    # stemming
    sentence_words = [stemmer.stem(word.lower()) for word in sentence_words]
    return sentence_words

# reprezentare binara bag-of-words
def bow(sentence, words):
    sentence_words = clean_up_sentence(sentence)
    # bag of words
    bag = [0]*len(words)  
    for s in sentence_words:
        for i,w in enumerate(words):
            if w == s: 
                bag[i] = 1

    return(np.array(bag))

In [17]:
ERROR_THRESHOLD = 0.25
def classify(sentence):
    # probabilitatile prezise de model
    results = model.predict([bow(sentence, words)])[0]
    # renuntam la intentiile cu probabilitate mica
    results = [[i,r] for i,r in enumerate(results) if r>ERROR_THRESHOLD]
    # sortam descrescator intentiile dupa probabilitate
    results.sort(key=lambda x: x[1], reverse=True)
    return_list = []
    for r in results:
        return_list.append((classes[r[0]], r[1]))
    # return perechi (intentie, probabilitate)
    return return_list

In [19]:
context = {}

def response(sentence, userID='123', show_details=False):
    results = classify(sentence)
    # daca avem macar o intentie valida, o procesam pe cea cu probabilitate maxima
    if results:
        # loop as long as there are matches to process
        while results:
            for i in intents['intents']:
                # cautam in dictionarul de intentii tagul returnat
                if i['tag'] == results[0][0]:
                    # daca intentia curenta asteapta un context pentru a fi valida, verificam ca contextul actual sa fie indeplinit
                    if (
                        not 'context_filter' in i
                        or (userID in context and 'context_filter' in i and i['context_filter'] == context[userID])
                    ):
                        if show_details:
                            print('tag:', i['tag'])
                        # daca intentia curenta actualizeaza contextul
                        if 'context_set' in i:
                            if show_details:
                                print('context:', i['context_set'])
                            context[userID] = i['context_set']
                        # returnam un raspuns aleator corespunzator intentiei
                        return random.choice(i['responses'])
            results.pop(0) # daca nu am putut da un raspuns pentru aceasta intentie, trecem la urmatoarea cu probabilitate maxima

    return "Sorry, I wasn't trained about this subject."  # nu a putut fi stabilita o intentie pentru fraza introdusa

print(response("hello there"))

Good to see you again. How are you?


In [20]:
try:
    while True:
        print(response(input("user: ")))
except KeyboardInterrupt:
    pass

user: Hi!
Good to see you again. How are you?
user: Fine, you?
Have a nice day
user: Thanks!
Happy to help!
